# Chapter 2 - Random Graph Models

In the first part of this notebook, we provide the code required to generate the Figures in Chapter 2 of the textbook.

In the second part, we consider the GitHub machine learning (ml) developers graph that we introduced in Chapter 1, and compare various statistics for this graph with the values we get for the random graphs models introduced in Chapter 2.

### Requirements

We use one new package in this notebook called ```powerlaw``` which can be installed via ```pip install powerlaw```.
Details and examples of use can be found here: https://arxiv.org/pdf/1305.0215.pdf.

As with the previous notebook, make sure to set the data directory properly in the next cell.


In [ ]:
datadir='../Datasets/'

In [ ]:
import igraph as ig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from collections import Counter
import powerlaw
from scipy.stats import poisson
from scipy.optimize import fsolve


# Part 1 - Figures for Chapter 2

## Figure 2.1: size of the giant component

We generate several binomial random graphs with $n$ nodes, where we vary the average node degree (thus, the number of edges). We consider $n=100$ below, and you can try for different $n$. Un-comment the second line to run with $n=10,000$ nodes as in the book (this will be much slower).

We plot the theoretical giant component size (black line) and the 90% confidence interval from the empirical data in grey, both as a function of the average degree; we see good agreement and we observe the various phases as described in the book. 

In [ ]:
n = 100
# n=10000 ## as in the book

gc_avg = []
gc_std = []

## range of values for average degree and number of repeats for each
avg_deg = np.arange(.1,10.1,.1)
Repeats = 1000

## generate random graphs and gather size of giant component
for deg in avg_deg:
    x = []
    p = deg/(n-1)
    for rep in range(Repeats):
        g = ig.Graph.Erdos_Renyi(n=n, p=p)
        x.append(g.connected_components().giant().vcount())
    ## average and standard deviation for a given average degree
    gc_avg.append(np.mean(x)) 
    gc_std.append(np.std(x))

## theoretical values
th_val = [np.log(n) for i in np.arange(.1,1.1,.1)] ## small values
def fn(x,d):
    return x+np.exp(-x*d)-1
for i in np.arange(1.1,10.1,.1):
    th_val.append(n*fsolve(fn,1,args=(i))[0])



In [ ]:
## plot empirical results (confidence intervals) and theoretical values
plt.fill_between(avg_deg,[x[0]-1.654*x[1] for x in zip(gc_avg,gc_std)],
                 [x[0]+1.645*x[1] for x in zip(gc_avg,gc_std)],color='lightgray')
plt.plot(avg_deg,th_val, color='black')
plt.suptitle('Random graph with '+str(n)+' nodes',fontsize=14)
plt.title('Theoretical predictions (black) vs empirical results (grey)',fontsize=12)
plt.xlabel('average degree',fontsize=14)
plt.ylabel('giant component size',fontsize=14);


## Figure 2.2: probability that the graph is connected

This is a similar experiment as above, but this time we look at the probability that the random graph is connected.
We vary some constant $c$ introduced in the book such that the edge probability for the binomial graphs is given by $(\log(n)+c)/n$. Once again we compare theory (black line) and experimental results (in grey) with $n=100$ nodes. Un-comment the second line to run with $n=10,000$ nodes as in the book (this will be much slower).

In the cell below, the grey area corresponds to a 90% confidence interval for proportions; for empirical proportion $x$ obtained from sample of size $n$, the formula is given by $x \pm 1.645 \sqrt{x(1-x)/n}$.

Here also we see good agreement between theory and experimental results.

In [ ]:
n = 100
#n = 10000

Repeats = 1000 ## number of repeats for each 'c' value

## set lower bound for the range of values for 'c'
lo = -int(np.floor(np.log(n)*10))/10
if lo<-10:
    lo = -10
c_range = np.arange(lo,10.1,.1)
ic_avg=[]

## loop over 'c' values
for c in c_range:
    x = []
    p = (c+np.log(n))/n
    for rep in range(Repeats):        
        g = ig.Graph.Erdos_Renyi(n=n, p=p)
        x.append(int(g.is_connected()))
    ic_avg.append(np.mean(x))

## theoretical values
th = [np.exp(-np.exp(-c)) for c in c_range]


In [ ]:
## plot
plt.fill_between(c_range,[x-1.654*np.sqrt(x*(1-x)/n) for x in ic_avg],
                 [x+1.645*np.sqrt(x*(1-x)/n) for x in ic_avg],color='lightgray')
plt.plot(c_range,th,color='black')
plt.suptitle('Random graph with '+str(n)+' nodes',fontsize=14)
plt.title('Theoretical predictions (black) vs empirical results (grey)',fontsize=12)
plt.xlabel(r'constant $c$',fontsize=14)
plt.ylabel('P(graph is connected)',fontsize=14);


## Figure 2.4: Distribution of shortest path lengths

We consider a series of binomial random graphs with expected average degree 5, where we vary the number of nodes from $n=50$ to $n=3,200$.

We see that as we double the number of nodes, the average shortest path lengths (in the giant component) increases slowly.


In [ ]:
sp_len = []
## number of nodes
n_range = [64,128,256,512,1024,2048]

for n in n_range:
    p = 5/(n-1)
    ## keep giant component
    g = ig.Graph.Erdos_Renyi(n=n, p=p).connected_components().giant()
    z = g.distances()
    sp_len.append([x for y in z for x in y if x>0])
    

In [ ]:
bins = np.arange(0.5,8.5,1)
fig, axs = plt.subplots(2, 3)
fig.suptitle('Shortest path length distribution')
for i in range(2):
    for j in range(3):
        axs[i,j].hist(sp_len[3*i+j], bins=bins, width=.9, density=True, color='darkgrey')
        axs[i,j].set_ylim(0,.48)
        axs[i,j].set_xticks([1,3,5,7])
        axs[i,j].set_title(str(n_range[3*i+j])+' nodes', fontsize=10)
        axs[i,j].set_xlabel('path length')
        axs[i,j].set_ylabel('proportion')
for ax in fig.get_axes():
    ax.label_outer()


## Figure 2.5 Poisson vs degree distributions

We plot the degree distribution for binomial random graphs with expected average degree 10, and $n=500$ nodes (the black dots), and we compare with the corresponding Poisson distribution (dashed line).

Try increasing $n$; the dots should get closer to the Poisson distribution.

Un-comment line 2 to run with $n=10,000$ as in the book.


In [ ]:
n = 500
#n = 10000 ## as in the book

p = 10/(n-1)
g = ig.Graph.Erdos_Renyi(n=n, p=p)
x = [x[0] for x in sorted(Counter(g.degree()).items())]
pmf = [poisson.pmf(k,10) for k in x]
frq = [x[1]/n for x in sorted(Counter(g.degree()).items())]
plt.plot(x,frq,'o',color='black')
plt.plot(x,pmf,':',color='black')
plt.xlabel('degree',fontsize=14)
plt.ylabel('frequency/pmf',fontsize=14);


## Figure 2.6 --  Power law graphs

We generate a random graph with $n=10,000$ nodes following power law degree distribution with exponent $\gamma=2.5$.
We do so using the Chung-Lu models described in section 2.5 of the book; we generate simple graphs (no loops or multiedges) and discard 0-degree nodes.

We then fit and plot the degree distribution of the obtained graph using the ```powerlaw``` package, see: https://arxiv.org/pdf/1305.0215.pdf



In [ ]:
## fast Chung-Lu: generate m distinct edges w.r.t. distribution d, no loops
def fast_CL(d, m):
    n = len(d)    ## number of nodes
    s = np.sum(d) 
    p = [i/s for i in d] ## we draw nodes w.r.t. degrees
    target = m ## number to generate
    tples = [] ## list of generated edges
    ## generate edges (tuples), drop collisions, until m edges are obtained.
    while len(tples) < target:
        s = target - len(tples) ## number left to generate
        e0 = np.random.choice(n, size=s, replace=True, p=p)
        e1 = np.random.choice(n, size=s, replace=True, p=p)
        tples.extend([(min(e0[i],e1[i]),max(e0[i],e1[i])) for i in range(len(e0)) if e0[i]!=e1[i]]) ## ignore loops
        tples = list(set(tples)) ## drop collisions
    return tples


In [ ]:
## power law graph
gamma = 2.5
n = 10000

## min and max degrees
delta = 1
Delta = np.sqrt(n)

## generate degrees
W = []
for i in np.arange(1,n+1):
    W.append(delta * (n/(i-1+n/(Delta/delta)**(gamma-1)))**(1/(gamma-1)))

# deg = [int(np.round(w)) for w in W] ## to enforce integer weights, not an obligation
deg = W

## generate graph with Chung-Lu model
m = int(np.mean(deg)*n/2)
tpl = fast_CL(deg,m)
g1 = ig.Graph.TupleList(tpl)

## number of isolated nodes (no edges)
iso = n-g1.vcount()
print('isolates:',iso)


In [ ]:
## run plfit and compute K-S statistic (details in the book)
d = g1.degree()
X = powerlaw.Fit(d)
print('\n',min(d),max(d))
print(X.power_law.alpha)
print(X.power_law.xmin)

In [ ]:
len(x)

In [ ]:
y = X.power_law.ccdf()
x = X.data
l = len(x)-1
plt.loglog(x,y,'.')
print('slope:',(np.log10(y[l])-np.log10(y[0]))/(np.log10(x[l])-np.log10(x[0])))

In [ ]:
X.xmin

In [ ]:
X.xmax

In [ ]:
fig1 = X.plot_ccdf(linewidth=2, color='b', original_data=False)
X.power_law.plot_ccdf(ax=fig1, color='b', linestyle=':');


In [ ]:
fig2 = X.plot_ccdf(linewidth=2, color='b', original_data=True)
#X.power_law.plot_pdf(color='b', linestyle='--', ax=fig2)


In [ ]:
plt.plot(X.xmins,X.Ds,'.');

In [ ]:
X.Ds[8]

In [ ]:
X.D

In [ ]:
R,p = X.distribution_compare('power_law','exponential') ## log-likelihood and p value
print(R,p)

In [ ]:
X.alphas

In [ ]:

plt.plot(X.alphas[:20],X.Ds[:20],'.');

In [ ]:
X.plot_ccdf();

## Figure 2.7: simple d-regular graphs

We generate several $d$-regular graphs and count how many are simple graphs.
We consider $d=2$ to $d=10$, with $n=100$ nodes. You can try for different $n$. Un-comment the second line to run with $n=10000$ nodes as in the book (this will be much slower).

We plot the empirical proportion of simple graphs below (black dots), and we compare with the theoretical values (dashed line). We see good agreement even for small value $n=100$.


In [ ]:
n = 100
# n = 10000
REP = 100
D = np.arange(2,11) 
simple = []
for d in D:
    x = 0
    for rep in range(REP):
        g = ig.Graph.Degree_Sequence([d for i in range(n)])
        x += int(g.is_simple())
    simple.append(x/REP)
th = [np.exp(-(d*d-1)/4) for d in D]

In [ ]:
plt.plot(D,simple,'o',color='black')
plt.plot(D,th,':',color='black')
plt.xlabel('degree',fontsize=14)
plt.ylabel('P(graph is simple)',fontsize=14);

# Part 2 -- Experiments section

We use the giant component of the GitHub machine learning (ml) developers subgraph that we introduced in Chapter 1. Recall this graph has 7,083 nodes and 19,491 edges. 

We compute several graphs statistics for this "base graph", as reported in the first column of Table 2.8 from the book.

We then generate random graphs with the same number of nodes and edges using 4 different models:
* binomial (only average degree)
* Chung-Lu (expected degree distribution)
* Configuration (exact degree distribution)
* Configuration with Viger method (connected, simple graph is obtained)

See section 2.8 of the book for a discussion of the results bit as a general observation, more complex models (such as the configuration model with Viger method) tend to preserve more characteristics of the reference graph.


In [ ]:
## read the GitHub edge list into a graph (gh)
D = pd.read_csv(datadir+'GitHubDevelopers/musae_git_edges.csv')
tuples = [tuple(x) for x in D.values]
gh = ig.Graph.TupleList(tuples, directed = False)

## Add some node features;
## There are 2 class of nodes
## 0: web developer (red), 1: ml developer (blue)
X = pd.read_csv(datadir+'GitHubDevelopers/musae_git_target.csv')
idx = [int(i) for i in gh.vs['name']]
sorterIndex = dict(zip(idx,range(len(idx))))
X['Rank'] = X['id'].map(sorterIndex)
X.sort_values(['Rank'], ascending=[True],inplace=True)
X.dropna(inplace=True)
gh.vs['target'] = list(X['ml_target'])
cls = ['grey','black']
gh.vs['color'] = [cls[i] for i in list(X['ml_target'])]
gh.es['color'] = 'grey'

## for github, 9739 are ml developers, build the subgraph
gh_ml = gh.subgraph([v for v in gh.vs() if v['color']=='black'])

## keep the giant component
sg = gh_ml.connected_components().giant()
print(sg.vcount(),'nodes and',sg.ecount(),'edges')

In [ ]:
## return statistics from Table 2.8 for graph G
def baseStats(G):
    deg = G.degree()
    return [G.vcount(),G.ecount(),np.min(deg),np.mean(deg),np.median(deg),np.max(deg),G.diameter(),
     np.max(G.connected_components().membership)+1,G.connected_components().giant().vcount(),sum([x==0 for x in G.degree()]),
     G.transitivity_undirected(),G.transitivity_avglocal_undirected()]
  

In [ ]:
## statistice for Base and random graphs
S = []
S.append(['Base Graph'] + baseStats(sg))
## Random (Erdos-Renyi) graph with same number of nodes and edges
er = ig.Graph.Erdos_Renyi(n=sg.vcount(), m=sg.ecount())
S.append(['Erdos-Renyi'] + baseStats(er))
## Random (Chung-Lu) graph with same degree distribution
tpl = fastCL(sg.degree(),sg.ecount()) 
cl = ig.Graph.Erdos_Renyi(n=sg.vcount(),m=0)
cl.add_edges(tpl)
S.append(['Chung-Lu'] + baseStats(cl))
## Random (configuration model) graph with same degree distribution
cm = ig.Graph.Degree_Sequence(sg.degree(),method='simple')
S.append(['Configuration'] + baseStats(cm))
## Random graph with same degree distribution using the
## configuration model with VL method, which yield a simple graph
cmvl = ig.Graph.Degree_Sequence(sg.degree(),method='vl')
S.append(['Configuration (VL)'] + baseStats(cmvl))


In [ ]:
## Store in dataframe and show results
D = pd.DataFrame(S,columns=['graph','nodes','edges',r'$d_{min}$',r'$d_{mean}$',
                             r'$d_{median}$',r'$d_{max}$','diameter','components','largest','isolates',
                             r'$C_{glob}$',r'$C_{loc}$'])
D = D.transpose()
D

### shortest path length distribution

We compute and compare the minimum path length distribution for several node pairs and for the 5 graphs we have (reference and 4 random ones). Sampling can be used to speed-up the process.

We consider the giant component for disconnected graphs.

We see a reasonably high similarity for all graphs, with the binomial random graph having slightly longer path lengths due to the absence of high degree (hub) nodes in that model.


In [ ]:
## sampling -- doing all vertices is slower
size = 1000

## using the giant component for disconnected graphs
er_g = er.connected_components().giant()
cl_g = cl.connected_components().giant()
cm_g = cm.connected_components().giant()

## to consider all vertices, replace the code below with:
# sp_sg = [i for v in sg.shortest_paths(source=None) for i in v]
# sp_er = [i for v in er_g.shortest_paths(source=None) for i in v]
# sp_cl = [i for v in cl_g.shortest_paths(source=None) for i in v]
# sp_cm = [i for v in cm_g.shortest_paths(source=None) for i in v]
# sp_cmvl = [i for v in cmvl.shortest_paths(source=None) for i in v]

# to use sampling:
## NB: we sample separately since we use the giant components and graphs may
##     have a different number of nodes (except the first and last one)
sp_sg = []
for v in np.random.choice(sg.vcount(),size=size,replace=False):
    sp_sg.extend(sg.distances(source=v)[0])
sp_er = []
for v in np.random.choice(er_g.vcount(),size=size,replace=False):
    sp_er.extend(er_g.distances(source=v)[0])
sp_cl = []
for v in np.random.choice(cl_g.vcount(),size=size,replace=False):
    sp_cl.extend(cl_g.distances(source=v)[0])
sp_cm = []
for v in np.random.choice(cm_g.vcount(),size=size,replace=False):
    sp_cm.extend(cm_g.distances(source=v)[0])
sp_cmvl = []
for v in np.random.choice(cmvl.vcount(),size=size,replace=False):
    sp_cmvl.extend(cmvl.distances(source=v)[0])

## generate boxplots   
plt.boxplot([sp_sg,sp_er,sp_cl,sp_cm,sp_cmvl],labels=['Base','Bin','CL','CM','CM(V)'],
            sym='.',whis=10, medianprops = dict(linestyle='-', linewidth=2.5,color='black'))
plt.ylabel('shortest path length',fontsize=14);

## save plot to file
#plt.savefig('pathlen_box.eps');

# Extra material

## More power law tests - GitHub subgraphs and Grid graph

We try to fit power law for degree distribution as we did before, this time for 3 real graphs:
* GitHub ml developers (giant component)
* GitHub web developers (giant component)
* Grid (Europe power grid graph, giant component)

While the first two exhibit power law degree distribution, this is clearly not the case for the Grid graph.


### GitHub ml subgraph

In [ ]:
## for github, 9739 are ml developers, build the subgraph
gh_ml = gh.subgraph([v for v in gh.vs() if v['color']=='black'])
## keep the giant component
sg = gh_ml.connected_components().giant()

## estimates for xmin and gamma
d = sg.degree()
X = powerlaw.Fit(d)

In [ ]:
fig1 = X.plot_ccdf(linewidth=2, color='b', original_data=False)
X.power_law.plot_ccdf(ax=fig1, color='b', linestyle=':');


In [ ]:
plt.plot(X.xmins,X.Ds,'.');

In [ ]:
print(X.power_law.alpha)
print(X.power_law.xmin)
print(X.power_law.D)


In [ ]:
R,p = X.distribution_compare('power_law','exponential') ## log-likelihood and p value
print(R,p)

### GitHub web subgraph

In [ ]:
## github web developers subgraph
gh_web = gh.subgraph([v for v in gh.vs() if v['color']!='black'])
## keep the giant component
sg = gh_web.connected_components().giant()

## estimates for xmin and gamma
d = sg.degree()
X = powerlaw.Fit(d)

In [ ]:
fig1 = X.plot_ccdf(linewidth=2, color='b', original_data=False)
X.power_law.plot_ccdf(ax=fig1, color='b', linestyle=':');


In [ ]:
plt.plot(X.xmins,X.Ds,'.');

In [ ]:
print(X.power_law.alpha)
print(X.power_law.xmin)
print(X.power_law.D)


In [ ]:
R,p = X.distribution_compare('power_law','exponential') ## log-likelihood and p value
print(R,p)

### Grid graph

In [ ]:
gr = ig.Graph.Read_Ncol(datadir+'GridEurope/gridkit_europe-highvoltage.edges', directed=False)
gr = gr.simplify()
## keep the giant component
sg = gr.connected_components().giant()

## estimates for xmin and gamma
d = sg.degree()
X = powerlaw.Fit(d)

In [ ]:
fig1 = X.plot_ccdf(linewidth=2, color='b', original_data=False)
X.power_law.plot_ccdf(ax=fig1, color='b', linestyle=':');


In [ ]:
print(X.power_law.alpha)
print(X.power_law.xmin)
print(X.power_law.D)


In [ ]:
X.power_law.KS()

## Independent sets

Illustrating a few functions to find independent sets.

In [ ]:
## generate random graph with (at least one) independent set 
## n: nodes, s: independent set size, d: avg degree
def indepSet(n,s,d):
    N = n-s
    di = n*d//2-s*d
    ## random graph with N nodes
    g = ig.Graph.Erdos_Renyi(n=N,m=di)
    ## extra nodes
    g.add_vertices(s)
    ## assign remaining degree to extra nodes
    z = np.random.choice(np.arange(N,n),size=s*d)
    deg = [x[1] for x in sorted(Counter(z).items())]
    for i in range(len(deg)):
        e = np.random.choice(N,deg[i],replace=False)
        for j in e:
            g.add_edge(j,i+N)
    p = list(np.random.permutation(n))
    G = g.permute_vertices(p)
    return G


In [ ]:
g = indepSet(50, 10, 20)

## every set of size min or mode
#ivs = g.independent_vertex_sets(min=9)

## largest set(s) only
ivs = g.largest_independent_vertex_sets()

## maximal sets (that can't be extended)
#ivs = g.maximal_independent_vertex_sets()

print(g.independence_number())

ivs